<a href="https://colab.research.google.com/github/yiboxu20/MachineLearning/blob/main/Resources/Module1/Decision_tree1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part of the notebook is based on Lope.ai.

In [1]:
%pylab inline
from IPython.display import Image
import numpy.linalg as LA

Populating the interactive namespace from numpy and matplotlib


([Top ten algorithms in data mining](http://www.cs.umd.edu/~samir/498/10Algorithms-08.pdf)): C4.5, $k$-Means, SVM,
Apriori, EM, PageRank, AdaBoost, $k$-NN, Naive Bayes, and CART

# Decision tree
- Defined by a hierarchy of rules (in form of a tree)

- Rules form the internal nodes. Each rule (internal node) tests the value of some property (feature) of the data.

- **root**: topmost internal node

- **leaf**: decision node at the ends

- **decision tree learning:** training data is used to construct/fit the decision tree model. Then the trained decision tree is used to predict for new input.

- **Advantage**: easy to interprete and quick prediction.



In [2]:
display(Image(url='https://github.com/yiboxu20/MachineLearning/blob/main/Resources/images/DT3.png?raw=true', width=700))

## Toy examples
- **Right:** Classification: 2-D points in blue vs. green region with 2 features
$(x_1, x_2)$.

- **Left**: The following decision tree is used to predict the region (blue/green) of a
new point in the order defined by the tree ( first $x_2$ and then $x_1$) This is also called **regression tree**.



The first node asks if $x_1$ is greater than
some threshold $t_1$. If yes, we then ask if $x_2$ is greater than some other threshold $t_2$. If yes, we enter the
bottom left leaf node. This corresponds to the region of space defined by
$R_1=\{\mathbf{x}: x_1\ge t_1, x_2\ge t_2\}$. The overall result is that we partition the 2d input space into 4 regions. By using enough splits (i.e., deep enough trees), we can make a piecewise linear approximationt to decision boundaries with more complex shapes, but it may require a lot of data to fit such a model.

The output for $R_1$ can be estimated using (if $y$ is continuous output as well)
\begin{align}
w_1 = \frac{\sum_{i=1}^Ny^{(i)}\mathbb{1}_{\mathbf{x}^{(i)}\in R_1} }{\sum_{i=1}^N \mathbb{1}_{\mathbf{x}^{(i)}\in R_1}}
\end{align}

Formally, the regression tree is defined by
\begin{align}
f(\mathbf{x}; \theta) = \sum_{j=1}^Jw_j \mathbb{1}_{\mathbf{x}\in R_j}
\end{align}
where $R_j$ is the region specified by the $j$’th leaf node, $w_j$ is the predicted output for that node, $J$ is the number of nodes. So the parameter is $\theta=\{(R_j, w_j)\}_{j=1}^J$.

In [3]:
display(Image(url='https://github.com/yiboxu20/MachineLearning/blob/main/Resources/images/DT1.png?raw=true', width=1000))

- Decide whether to play or not to play Tennis based on 4 features:
outlook, temperature, humidity, wind.

  For categorical inputs, we can define the splits based
on comparing feature to each of the possible values for that feature, like "outlook" having three possible value, 'sunny', 'overcase' and 'rain'.

- **Question:** why does it make more sense to test the "outlook"
feature  first? "Temperature" is completely ignored?
   - Because "outlook" is the most informative of all the 4 features and "temperature" is not informative.
   
   - We need to quantify the informativeness.




In [4]:
display(Image(url='https://github.com/yiboxu20/MachineLearning/blob/main/Resources/images/DT.png?raw=true', width=1000))

## Decision tree construction

- Data comes in records of the form $\{\mathbf{x}^{(i)}, y^{(i)}\}_{i=1}^N$, the vector $\mathbf{x}$ is composed of **the features variables**, $x_1, x_2, x_3$, can be continuous or discrete. The dependent variable $y$, is the **target variable** that we are trying to understand, classify or generalize.

- We use training data $\{\mathbf{x}^{(i)}, y^{(i)}\}_{i=1}^N$ to construct the decision tree. Each internal node corresponds to a rule to test some feature.


- To fit the model, we have the loss function
$$ \mathcal{L}(\theta)=\sum_{i=1}^N\ell(y^{(i)}, f(\mathbf{x}^{(i)};\theta)) =\sum_{j=1}^J \sum_{\mathbf{x}^{(i)}\in R_j}\ell(y^{(i)},w_j)$$
 Unfortunately, this is not differentiable, because of the need to learn the discrete tree structure.

- **Idea:** Use a greedy procedure, in which we iteratively grow the tree one node at a time.
Highly informative features are placed higher up in the tree;
will need a way to rank features according to their information
content.

We will introduce two decision tree algorithms: **Iterative Dichotomiser 3(ID3)** and **Classification and regression trees (CART)**.

# Iterative Dichotomiser 3
In ID3, we will use **entropy** and **information gain** as the criteria/metric for ranking. All features are **categorical/discrete** features and the output are binary($+$ or $-$).  


## Entropy

- Entropy (in information theory) measures the
randomness/uncertainty in the data

- Given a set of $\mathcal{S}$ with $k$ classes, associated with a Multinomial probability distribution $\mathbf{p}=(p_1, \dots, p_k)$ with $\sum_{i}p_i =1$.

- Entropy is defined as follows
  $$ H(\mathcal{S}) = -\sum_{i=1}^k p_i\log_2 p_i$$
  Note the logarithm is base 2 instead of base $e$.

  Example: for $k=2$, $\mathcal{S}_1$ with $\mathbf{p}=(1/2, 1/2)$; $\mathcal{S}_2$ with $\mathbf{p}=(0.3, 0.7)$. Then $H(\mathcal{S}_1)=1$ and $H(\mathcal{S}_2)=-0.3\log_2(0.3)-0.7\log_2(0.7)\approx 0.8816$.

**Property:**

- $0\le H(\mathcal{S})\le \log_2(k)$. The minimum is reached when $p_i=1$ for some class $i$ (no uncertainty, since all instances are from the same class). The maximum is reached when $p_i=\frac{1}{k}$ for all classes
(uniform distribution).

- In general,

  Dominant/skewed classes $\rightarrow$ Low entropy/uncertainty.      
  
  Equi-probable classes $\rightarrow$ High entropy/uncertainty.




## Information gain

- **Information gain (IG)** of set $\mathcal{S}$ on some feature $F$ is
$$ \text{IG}(\mathcal{S}; F)= H(\mathcal{S})- H(\mathcal{S}|F)$$
  where $H(\mathcal{S}|F)$ is conditional entropy of $\mathcal{S}$ given the value of feature $F$.

- $\text{IG}(\mathcal{S}; F)$ measures the decrease in uncertainty about $\mathcal{S}$ once the value of feature $F$ is known.


- Specifically, conditional entropy of $\mathcal{S}$ on $F$ is computed by
$$ H(\mathcal{S}|F) = \sum_{f\in F} P(F=f)H(\mathcal{S}_f)=\sum_{f\in F} \frac{|\mathcal{S}_f|}{|\mathcal{S}|}H(\mathcal{S}_f)$$
  where $\mathcal{S}_f$ is the subset of $\mathcal{S}$ with feature $F$ having value $f$, $ |\cdot|$ denotes the number of elem of a set.




### Computing information gain: Tennis example

- $\mathcal{S}=\{1, \dots, 14\}$, with 9 "play" and 5 "no-play". Then
   $$H(\mathcal{S})= -(9/14)\log_2(9/14)-(5/14)\log_2(5/14)\approx 0.94 $$

- Consider the feature $F=$wind. Then $f\in\{\text{strong}, \text{weak}\}$.
$$\mathcal{S}_{\text{strong}} = \{2,6,7,11,12,14\}, \ \text{play}=\{3+, 3-\}, \ H(\mathcal{S}_{\text{strong}})=1 $$

$$\mathcal{S}_{\text{weak}} = \{1,3,4,5,8,9,10,13\}, \ \text{play}=\{6+, 2-\}, \ H(\mathcal{S}_{\text{weak}})\approx 0.811 $$

- information gain on the feature "wind":
\begin{align}
 \text{IG}(\mathcal{S},\text{wind})&= H(\mathcal{S}) - H(\mathcal{S}|\text{wind}) \\
 &= H(\mathcal{S}) -\frac{|\mathcal{S}_{\text{weak}}|}{|\mathcal{S}|}H(\mathcal{S}_{\text{weak}}) -\frac{|\mathcal{S}_{\text{strong}}|}{|\mathcal{S}|}H(\mathcal{S}_{\text{strong}}) \\
 & =0.94 -8/14\cdot0.811 - 6/14 \cdot 1=0.048
\end{align}


- Similarly,
\begin{align}
&\text{IG}(\mathcal{S},\text{outlook}) = 0.246 \\
&\text{IG}(\mathcal{S},\text{humidity}) = 0.151 \\
&\text{IG}(\mathcal{S},\text{temperature)}) = 0.029
\end{align}

- '"outlook" has most IG: chosen as the first feature to test
(root node)

In [5]:
display(Image(url='https://github.com/yiboxu20/MachineLearning/blob/main/Resources/images/tennis.png?raw=true', width=400))
display(Image(url='https://github.com/yiboxu20/MachineLearning/blob/main/Resources/images/tennis2.png?raw=true', width=400))

### Grow the tree

Iteratively select the feature with highest IG for each child node to
grow the tree.

For **left node:**
-  $\mathcal{S}=\{1,2,8,9,11\}$ with play=$\{2+, 3-\}$,
$$H(\mathcal{S}) = -(2/5)\log_2(2/5)-(3/5)\log_2(3/5)\approx 0.97$$

- for the feature "temperature":
  - $\mathcal{S}_{\text{hot}} = \{1,2\}$ with play=$\{0+, 2-\}$, $H(\mathcal{S}_{\text{hot}})=0$.

  - $\mathcal{S}_{\text{mild}} = \{8,11\}$ with play=$\{1+, 1-\}$, $H(\mathcal{S}_{\text{mild}})=1$.

  - $\mathcal{S}_{\text{cool}} = \{9\}$ with play=$\{1+, 0-\}$, $H(\mathcal{S}_{\text{cool}})=0$.


- information gain on the feature "temperature":
\begin{align}
\text{IG}(\mathcal{S}, \text{temp}) &= H(\mathcal{S})-H(\mathcal{S}|\text{temp}) \\
&=H(\mathcal{S}) -\frac{|\mathcal{S}_{\text{hot}}|}{|\mathcal{S}|}H(\mathcal{S}_{\text{hot}}) -\frac{|\mathcal{S}_{\text{mild}}|}{|\mathcal{S}|}H(\mathcal{S}_{\text{mild}}) -\frac{|\mathcal{S}_{\text{cool}}|}{|\mathcal{S}|}H(\mathcal{S}_{\text{cool}}) \\
&=0.97-2/5\cdot 0 - 2/5\cdot 1 - 1/5\cdot 0= 0.57
\end{align}

- Similarly,
 $$ \text{IG}(\mathcal{S}, \text{humidity})=0.97 $$
 $$  \text{IG}(\mathcal{S}, \text{wind})=0.019$$
 Choose feature "humidity" for the left node.



For **middle node:**
 - $\mathcal{S}=\{3,7,12,13\}$ with play=\{4+, 0-\}: no
need to grow due to identical labels ($H(\mathcal{S})=0$)

For **right node:**

 - Compute IG for each feature except for
"outlook", choose "wind" since it has the highest IG.

End up with a decision tree perfectly fitting the training data

For **Stopping criteria**: stop expanding a node further when
  - all training examples have the same label
  - run out of features to test (leading to a training error)


## ID3 algorithm
**Iterative Dichotomiser 3(ID3)**:

- Compute the entropy for dataset

- For every feature:
       1.calculate entropy for all categorical values
       2.take average information entropy for the current feature
       3.calculate gain for the current feature
- Pick the highest gain feature.

- Repeat until we get the tree we desired.


# Let's code this up.
For this portion, I will use some basic functions in pandas. It is possible that you can code this up without using it.

In [6]:
import pandas as pd
eps  = 10**-12

In [7]:
outlook = 'sunny,sunny,overcast,rainy,rainy,rainy,overcast,sunny,sunny,rainy,sunny,overcast,overcast,rainy'.split(',')
temp = 'hot,hot,hot,mild,cool,cool,cool,mild,cool,mild,mild,mild,hot,mild'.split(',')
humidity = 'high,high,high,high,normal,normal,normal,high,normal,normal,normal,high,normal,high'.split(',')
windy = 'weak,strong,weak,weak,weak,strong,strong,weak,weak,weak,strong,strong,weak,strong'.split(',')
play = 'no,no,yes,yes,yes,no,yes,no,yes,yes,yes,yes,yes,no'.split(',')

dataset ={'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])
df


,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes
5,rainy,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rainy,mild,normal,weak,yes


In [8]:
df.loc[0]

,0
outlook,sunny
temp,hot
humidity,high
windy,weak
play,no


In [9]:
df[df['outlook']=='sunny'].reset_index(drop=True)

,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,sunny,mild,high,weak,no
3,sunny,cool,normal,weak,yes
4,sunny,mild,normal,strong,yes


In [10]:
#step 1: Compute the entropy for data-set

def entropy(df): #H(S)
    target = df.keys()[-1]
    entropy_data = 0
    target_values = df[target].unique() #yes or no
    for target_value in target_values:
        fraction = df[target].value_counts()[target_value]/len(df[target])
        entropy_data += -fraction*np.log2(fraction)
    return entropy_data

print('entropy for the data set is',entropy(df) )

entropy for the data set is 0.9402859586706311


In [11]:
# define a function ent to calculate conditional entropy of each feature
def entropy_feature(df,feature): #H(S|F)
    target = df.keys()[-1]
    target_values = df[target].unique()  #This gives all 'Yes' and 'No'
    variables = df[feature].unique()    #This gives different features (f values)

    entropy = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_values:
            num = len(df[feature][df[feature]==variable][df[target] ==target_variable]) #numerator
            den = len(df[feature][df[feature]==variable])  #denominator
            fraction = num/(den+eps)  #+eps can prevent runtime error of divide 0.
            entropy_each_feature += -fraction*log2(fraction+eps) #This calculates entropy for one feature   H(S_f)
        fraction2 = den/len(df) # P(F=f)
        entropy += -fraction2*entropy_each_feature   #Sums up all the entropy, H(S|F)=\sum P(F=f) H(S_f)

    return(abs(entropy))


In [12]:
print('features are', df.keys()[:-1])
entropy_feature_list = {k:entropy_feature(df,k) for k in df.keys()[:-1]}
print('entropy of each feature is', entropy_feature_list)

entropy_feature_list['temp']

features are Index(['outlook', 'temp', 'humidity', 'windy'], dtype='object')
entropy of each feature is {'outlook': np.float64(0.6935361388938892), 'temp': np.float64(0.9110633930089052), 'humidity': np.float64(0.7884504573054976), 'windy': np.float64(0.892158928259553)}


np.float64(0.9110633930089052)

In [13]:
# calculate Info gain of each feature
def ig(df):
    IG = []
    for feature in df.keys()[:-1]:
      IG.append(entropy(df)-entropy_feature(df,feature))
    return IG


print('IG of each feature is', ig(df))

IG of each feature is [np.float64(0.24674981977674193), np.float64(0.029222565661725985), np.float64(0.15183550136513357), np.float64(0.04812703041107813)]


# Let's build the decision tree

In [14]:

def get_subtable(df, node,variable):
  return df[df[node] == variable].reset_index(drop=True)


def buildTree(df,tree=None):
    target = df.keys()[-1]   #To make the code generic, changing target variable class name
    features = df.keys()[:-1]
    #Here we build our decision tree

    #Get feature with maximum information gain
    IG = ig(df)
    node = features[argmax(IG)]

    #Get distinct value of that feature
    variables = df[node].unique()

    #Create an empty dictionary to create tree
    if tree is None:
        tree={}
        tree[node] = {}

   #We make loop to construct a tree by calling this function recursively.
    #In this we check if the subset is pure and stops if it is pure.

    for variable in variables:

        subtable = get_subtable(df,node,variable)
        clValue,counts = unique(subtable[target],return_counts=True)

        if len(counts)==1:#Checking purity of subset
            tree[node][variable] = clValue[0]
        else:
            tree[node][variable] = buildTree(subtable) #Calling the function recursively

    return tree

In [15]:
import pprint
t=buildTree(df)
pprint.pprint(t)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}


In [16]:
df1=pd.concat([df, pd.DataFrame([['sunny','hot','normal','strong','no']], columns=['outlook','temp','humidity','windy','play'])],ignore_index=True)
# append no longer supported in DataFrames
df1

,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes
5,rainy,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rainy,mild,normal,weak,yes


In [17]:

t1=buildTree(df1)
pprint.pprint(t1)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'temp': {'cool': 'yes',
                                'hot': 'no',
                                'mild': {'humidity': {'high': 'no',
                                                      'normal': 'yes'}}}}}}


- This extra complexity may not be worth it; may lead to
overfitting if the test data follows the same pattern as normal
training data

Solution:
- Retain a validation set from the original training set.

- Pruning to avoid overfitting: remove node from bottom to up
(in a greedy manner) if the removal improves the validation
accuracy, until it starts worsening.

In [18]:
url = "https://raw.githubusercontent.com/yiboxu20/MachineLearning/refs/heads/main/Resources/data/credithistory.csv"
c = pd.read_csv(url)
c= c[['collateral','income','debt','credithistory','risk']]
c

,collateral,income,debt,credithistory,risk
0,none,$0to$15K,high,bad,high
1,none,$15Kto$35K,high,unknown,high
2,none,$15Kto$35K,low,unknown,moderate
3,none,$0to$15K,low,unknown,high
4,none,over$35K,low,unknown,low
5,adequate,over$35K,low,unknown,low
6,none,$0to$15K,low,bad,high
7,adequate,over$35K,low,bad,moderate
8,none,over$35K,low,good,low
9,adequate,over$35K,high,good,low


In [19]:
t1=buildTree(c)
pprint.pprint(t1)

{'income': {'$0to$15K': 'high',
            '$15Kto$35K': {'credithistory': {'bad': 'high',
                                             'good': 'moderate',
                                             'unknown': {'debt': {'high': 'high',
                                                                  'low': 'moderate'}}}},
            'over$35K': {'credithistory': {'bad': 'moderate',
                                           'good': 'low',
                                           'unknown': 'low'}}}}
